### RA - Job History Coding 
Created a long dataset to record the individuals' occupation history including the jobs and the gaps using UKB online follow-up data. For every individual, all his/her job histories are recorded in sequential years, where each row represents one specific year in his/her career life. The years for the gaps are not included in the timeline, but are recorded independently using additional columns, categorized by the gap reasons.

In [13]:
import fnmatch

In [14]:
import os 
import pandas as pd
import numpy as np
import datetime
import fnmatch
import warnings
warnings.filterwarnings('ignore')
os.chdir('C:\\Users\\chuyi\\Desktop\\NUS RA\\Recode the job history')

#### 1. Loaded the data 
Cut from subset_online_follow_up.txt

In [15]:
large = pd.read_table('full_dataset_cut4.txt')
#large = pd.read_table('subset_online_follow_up.txt')
large = large.rename(columns={'f.eid': 'ID',
                              'f.22200.0.0' : 'Year of Birth',
                              'f.22599.0.0' : 'Number of Jobs Held'})
large[large['Year of Birth'] > 0].shape

(515, 1252)

#### 2. Create the dataframe for job history with job coding

In [16]:
items = ['f.22602.0.*', 'f.22603.0.*', 'f.22617.0.*'] 

# The function to create the long data with selected columns 
# input: items (using wildcard * to match the field index

def create_long_data(items) :

    info = ['ID','Year of Birth', 'Number of Jobs Held']
    col_list = large.columns.tolist()
    select = fnmatch.filter(col_list, 'f.22601.0.*')
    df_job = large.loc[:, info + select]
    df_ind = pd.wide_to_long(df_job, 
                             stubnames = 'f.22601.0.', 
                             i = info, 
                             j = 'Index').reset_index().dropna()
 
    to_match = items
    to_match2 = to_match.copy()
    for i in list(range(0, len(to_match))) :
        to_match2[i] = to_match[i][:-1]
    
    for i in list(range(0, len(to_match))) :
        select = fnmatch.filter(col_list, to_match[i])
        df = large.loc[:, info + select]
        df_step = pd.wide_to_long(df, stubnames = to_match2[i], i = info, j = 'Index').reset_index().dropna()
        df_ind = df_ind.merge(df_step, on = info + ['Index'], how = 'left')


    df_ind = df_ind.rename(columns={'f.22601.0.' : 'Job Code',
                                    'f.22602.0.' : 'Job Start Year',
                                    'f.22603.0.' : 'Job End Year',
                                    'f.22617.0.' : 'SOC2000'})
    df_ind['Job End Year'] = df_ind['Job End Year'].replace(-313, 2017)
    df_ind['Job Hold Year'] = df_ind['Job End Year'] - df_ind['Job Start Year'] + 1
    df_ind['Year'] = df_ind['Job Start Year']

    # duplicate the rows by their working years 
    df_long = df_ind.loc[df_ind.index.repeat(df_ind['Job Hold Year'])].reset_index(drop=True)
    

    # change the years and create the long data with sequential years
    id_list = df_long['ID'].unique()
    accumulate = pd.DataFrame(columns=df_ind.columns)
    for i in list(range(0, len(id_list))):
        sub1 = df_ind[df_ind['ID'] == id_list[i]]
        index_list = list(range(0,len(sub1)))
        for j in index_list:
            sub2 = df_long[(df_long['ID'] == id_list[i]) & (df_long['Index'] == j)]  
            sub2.loc[:,'Year'] = np.arange(sub2.iloc[0,5], sub2.iloc[0,5] + len(sub2))
            accumulate = accumulate.append(sub2)
            #accumulate = accumulate.concat(sub2)
            accumulate = pd.concat([accumulate, sub2], ignore_index=True)

    accumulate['Age'] = accumulate['Year'] - accumulate['Year of Birth']
    accumulate['Job End Year'] = accumulate['Job End Year'].replace(2017, -313) 
    accumulate['Index'] = accumulate['Index'] + 1
    accumulate = accumulate.drop_duplicates(['ID','Year','Age'],keep='last')
    accumulate.to_excel('long data test3.xlsx')
    return accumulate


# run the function
df_accumulate = create_long_data(items)

#### 3. Create the dataframe that collects all the gap records for individuals

In [17]:
# create the dataframe for gap years

df_gaps = large.loc[:, ['ID', 'f.22661.0.0']].rename(columns={'f.22661.0.0':'Number of Gaps'})
df_gaps['Number of Gaps'] = df_gaps['Number of Gaps'].replace(np.nan, 0)

col_list = large.columns.tolist()
gap_start_col = fnmatch.filter(col_list, 'f.22663.0.*')
gap_end_col = fnmatch.filter(col_list, 'f.22664.0.*')
gap_coding_col = fnmatch.filter(col_list, 'f.22660.0.*')

df_gap_start =  large.loc[:, ['ID','Year of Birth'] + gap_start_col]
df_step1 = pd.wide_to_long(df_gap_start, stubnames = 'f.22663.0.', i = ['ID', 'Year of Birth'], j = 'Index').reset_index().dropna()
df_gap_end =  large.loc[:, ['ID','Year of Birth'] + gap_end_col]
df_step2 = pd.wide_to_long(df_gap_end, stubnames = 'f.22664.0.', i = ['ID', 'Year of Birth'], j = 'Index').reset_index().dropna()
df_gap_coding =  large.loc[:, ['ID','Year of Birth'] + gap_coding_col]
df_step3 = pd.wide_to_long(df_gap_coding, stubnames = 'f.22660.0.', i = ['ID', 'Year of Birth'], j = 'Index').reset_index().dropna() 

df_full_gap = df_step1.merge(df_step2, on = ['ID', 'Year of Birth', 'Index'], how='outer').merge(df_step3, on = ['ID', 'Year of Birth', 'Index'], how='outer')

df_full_gap = df_full_gap.rename(columns={'f.22663.0.': 'Gap Start Year',
                                          'f.22664.0.': 'Gap End Year',
                                          'f.22660.0.' : 'Reason of Gap'})

df_full_gap2 = df_full_gap.drop_duplicates(['ID', 'Gap Start Year'], keep='last')
df_full_gap['Length of Gap'] = df_full_gap['Gap End Year'] - df_full_gap['Gap Start Year'] + 1
df_full_gap.loc[df_full_gap['Length of Gap'] < 0, 'Length of Gap'] = np.nan
df_full_gap['Gap Before This Job'] = "Yes"   
#df_full_gap 


#### 4. Reconstruct the gap information and merge it to the job history dataframe 

##### 4.1 Identify the retirement year and add a column 'Retired Year'

In [18]:
# identify the retired year 
df_retired = df_full_gap[(df_full_gap['Gap End Year'] == -313) & (df_full_gap['Reason of Gap'] == 108)].drop_duplicates(['ID'], keep = 'last')
df_retired = df_retired[['ID', 'Year of Birth', 'Gap Start Year']].rename(columns={'Gap Start Year' : 'Retired Year'})
df_add_gap1 = df_accumulate.merge(df_retired, on = ['ID', 'Year of Birth'], how = 'left')

##### 4.2 Find the gaps that before and closest to the job that recorded in the current row

In [19]:
# match job start year with gap end year 
df_full_gap['Gap End Year2'] = df_full_gap['Gap End Year']
df_full_gap2 = df_full_gap.drop(['Index'], axis=1).rename(columns={'Gap End Year2' : 'Job Start Year'})    # match gap end year with job start year 
df_add_gap2 = df_add_gap1.merge(df_full_gap2, on = ['ID','Year of Birth', 'Job Start Year'], how = 'left')

# some additional cases: match job start year with gap end year + 1
df_full_gap['Gap End Year3'] = df_full_gap['Gap End Year'] + 1
df_full_gap_adj = df_full_gap.drop(['Index'], axis=1).rename(columns={'Gap End Year3' : 'Job Start Year'})    # match gap end year with job start year 
df_add_gap_adj = df_add_gap1.merge(df_full_gap_adj, on = ['ID','Year of Birth', 'Job Start Year'], how = 'left')

# merge two dataframes 
combine_col_list = ['Gap Before This Job', 'Gap Start Year', 'Gap End Year', 'Reason of Gap', 'Length of Gap']
for col_name in combine_col_list :
    df_add_gap2[col_name] = df_add_gap2[col_name].fillna(df_add_gap_adj.pop(col_name))   ########## adjust

# merge df with number of gaps column
df_add_gap2 = df_add_gap2.merge(df_gaps, on = ['ID'], how = 'left')


##### 4.3 Check if the job held after the retirement, and add the column 'Is After Retirement'.

In [20]:
# add the column "the job is after retirement"
df_after_retirement = df_add_gap2[(df_add_gap2['Gap Before This Job'] == "Yes") & (df_add_gap2['Reason of Gap'] == 108)].drop_duplicates(['ID','Index'])
df_after_retirement['Is After Retirement'] = "Yes"
df_after_retirement = df_after_retirement[['ID', 'Job Code', 'Index', 'Gap Start Year', 'Gap End Year', 'Reason of Gap', 'Is After Retirement']]
df_add_gap3 = df_add_gap2.merge(df_after_retirement, on = ['ID', 'Job Code', 'Index', 'Gap Start Year', 'Gap End Year', 'Reason of Gap'], how = 'left')

col_order = ['ID', 'Year of Birth', 'Number of Jobs Held', 'Number of Gaps', 'Retired Year',
             'Job Code', 'SOC2000', 'Year', 'Age',  'Job Start Year', 'Job End Year', 'Index', 
             'Is After Retirement', 'Gap Before This Job', 'Gap Start Year', 'Gap End Year', 'Reason of Gap', 'Length of Gap']
df_add_gap3 = df_add_gap3[col_order]

##### 4.4 Detected the gaps that can't be captured by matching the column (no job followed)
Usually the case when someone reported several different gaps (with different reasons) between two jobs. 

In [21]:
# get the undetected gaps: 
detected_gap = df_add_gap3[df_add_gap3['Gap Start Year'].notna()].drop(['Year', 'Age'], axis = 1).drop_duplicates()
full_gap = df_full_gap[['ID', 'Gap Start Year', 'Gap End Year', 'Reason of Gap']]

merge = full_gap.merge(detected_gap, on = ['ID', 'Gap Start Year', 'Gap End Year', 'Reason of Gap'], how = 'left')
undetected = merge[merge['Year of Birth'].isnull()]
undetected.to_excel('detect the gap1.xlsx')

undetected = undetected[(undetected['Gap End Year'] != -313) | (undetected['Reason of Gap'] != 108)][['ID', 'Gap Start Year', 'Gap End Year', 'Reason of Gap']]
undetected_gap = undetected[undetected['Gap End Year'] != -313]
undetected_gap.to_excel('detect the gap2.xlsx')

# get the last recorded job end year (except -313: currently working on)
df_last_job_end = df_accumulate[df_accumulate['Job End Year'] != -313].groupby(['ID'])['Job End Year'].max().reset_index()
mid = undetected_gap.merge(df_last_job_end, on = ['ID'], how = 'left')

# the finalized undetected gap 
undetected_ava_gap = mid[mid['Gap Start Year'] < mid['Job End Year']].drop(['Job End Year'], axis=1)

##### 4.5 Code these undetected gaps using additional columns for each gap reasons 

In [22]:
df_add_gap4 = df_add_gap3.copy()

# make a function to code the gaps below
reason_list = [101, 102, 103, 105, 106, 107, -717]
reason_str_list = ['Part-time Gap','Voluntary Work Gap', 'Education Gap', 'Family Reason Gap', 'Sickness Gap', 'Unemployment Gap', 'Other Reason Gap']
start_col_list = ['P_Start', 'V_Start', 'E_Start', 'F_Start', 'S_Start','U_Start', 'O_Start']
end_col_list = ['P_End', 'V_End', 'E_End', 'F_End', 'S_End', 'U_End', 'O_End']

for i in range(0,len(reason_list)):

    for_spec = undetected_ava_gap[undetected_ava_gap['Reason of Gap'] == reason_list[i]]
    id_list = for_spec['ID']
    current = df_accumulate.loc[df_accumulate['ID'].isin(id_list)].drop(['Year', 'Age'], axis = 1).drop_duplicates()
    current2 = current.merge(for_spec , on = ['ID'], how = 'left')
    keep_spec = current2[current2['Gap End Year'] <= current2['Job Start Year']].drop_duplicates(subset=['ID', 'Gap Start Year', 'Gap End Year'], keep = 'first')
    keep_spec = keep_spec.drop_duplicates(subset = ['ID','Job Start Year', 'Job End Year'], keep = 'last').rename(columns={'Gap Start Year' : start_col_list[i],
                                                                                                                             'Gap End Year' : end_col_list[i]})
    keep_spec[reason_str_list[i]] = "Yes"
    keep_spec = keep_spec[['ID','Job Start Year', 'Job End Year', start_col_list[i], end_col_list[i], reason_str_list[i]]]

    df_add_gap4 = df_add_gap4.merge(keep_spec, on = ['ID','Job Start Year', 'Job End Year'], how = 'left')


for i in (list(range(0,len(df_add_gap4)))) :

    for j in range(0,len(reason_list)) : 

        if df_add_gap4['Reason of Gap'][i] == reason_list[j] :
            df_add_gap4[reason_str_list[j]][i] = "Yes"
            df_add_gap4[start_col_list[j]][i] = df_add_gap4['Gap Start Year'][i]
            df_add_gap4[end_col_list[j]][i] = df_add_gap4['Gap End Year'][i]


##### 4.6 For each individual, create the column 'Have Gap During Career' 
To identify if the individual had the gaps in his/her entire career life (except the retirement).

In [23]:
df_add_gap4['Retired Year'] = df_add_gap4['Retired Year'].replace(np.nan, -313)
df_add_gap4['Have Gap During Career'] = df_add_gap4['Is After Retirement'].replace("Yes", np.nan)

for i in range(0, len(df_add_gap4)) :
    if df_add_gap4['Number of Gaps'][i] == 0:
        df_add_gap4['Have Gap During Career'][i] = "No"
    if (df_add_gap4['Number of Gaps'][i] == 1) & (df_add_gap4['Retired Year'][i] != -313):
        df_add_gap4['Have Gap During Career'][i] = "No"

df_add_gap4['Have Gap During Career'] = df_add_gap4['Have Gap During Career'].replace(np.nan,"Yes")

# update the column 'Gap Before This Job'
for i in range(0, len(df_add_gap4)):
    if (df_add_gap4['Part-time Gap'][i] == "Yes") | (df_add_gap4['Voluntary Work Gap'][i] == "Yes") | (df_add_gap4['Education Gap'][i] == "Yes") | (df_add_gap4['Family Reason Gap'][i] == "Yes") | (df_add_gap4['Sickness Gap'][i] == "Yes") | (df_add_gap4['Unemployment Gap'][i] == "Yes") |(df_add_gap4['Other Reason Gap'][i] == "Yes"):
        df_add_gap4['Gap Before This Job'][i] = "Yes"


##### 4.7 Organize the columns and finalize the job history dataframe 

In [24]:
col_order2 = ['ID', 'Year of Birth', 'Number of Jobs Held', 'Have Gap During Career', 'Number of Gaps', 'Retired Year',
             'Job Code', 'SOC2000', 'Year', 'Age',  'Job Start Year', 'Job End Year', 'Index', 
             'Is After Retirement', 'Gap Before This Job', 'Gap Start Year', 'Gap End Year', 'Reason of Gap', 'Length of Gap',
             'Part-time Gap', 'P_Start', 'P_End', 'Voluntary Work Gap', 'V_Start', 'V_End', 
             'Education Gap', 'E_Start', 'E_End', 'Family Reason Gap', 'F_Start', 'F_End',
             'Sickness Gap', 'S_Start', 'S_End', 'Unemployment Gap', 'U_Start', 'U_End',
             'Other Reason Gap', 'O_Start', 'O_End']
df_add_gap4 = df_add_gap4[col_order2]

df_add_gap4.to_excel('add_gap4.xlsx')

### Link SOC2000 with ONET coding

In [25]:
onet = pd.read_csv('uksoc_onet_all.csv')

In [26]:
col_list = onet.columns.tolist()
col_list

['jrecord',
 'leadership',
 'leadership_d',
 'jcomplexity_IM',
 'jcomplexity_LV',
 'ocomplexity_IM',
 'ocomplexity_LV',
 'interdep',
 'autonomy',
 'routinization',
 'ExtreEnviron',
 'hazard',
 'ImpDecis',
 'ConseErr',
 'RespFHealth',
 'structure',
 'FreeTDecis',
 'IndepComplet',
 'AttentTDetail',
 'SocialSkill',
 'Competition',
 'Innovation',
 'DealWPeop',
 'jdemand_info_IM',
 'jdemand_info_LV',
 'jdemand_manage_IM',
 'jdemand_manage_LV',
 'jdemand_emotion_IM',
 'jdemand_emotion_LV',
 'jdemand_physical_IM',
 'jdemand_physical_LV',
 'skill_DM_IM',
 'skill_DM_LV',
 'skill_IP_IM',
 'skill_IP_LV',
 'wactivity_DM_IM',
 'wactivity_DM_LV',
 'wactivity_IP_IM',
 'wactivity_IP_LV',
 'wstyle_DM',
 'wstyle_IP',
 'creatIM',
 'creatLV',
 'creatIM_short',
 'auto_rank',
 'auto_probability',
 'auto_label',
 'job_zone',
 'abi_verb_IM',
 'abi_verb_LV',
 'abi_idea_IM',
 'abi_idea_LV',
 'abi_quan_IM',
 'abi_quan_LV',
 'abi_mem_IM',
 'abi_mem_LV',
 'abi_percep_IM',
 'abi_percep_LV',
 'abi_spac_IM',
 'abi_sp

In [27]:
tolink = onet[['jrecord', 'leadership', 'leadership_d','jdemand_manage_LV']].rename(columns = {'jrecord' : 'SOC2000'})

In [28]:
df_linked_leadership = df_add_gap4.merge(tolink, on = ['SOC2000'], how = 'left')

In [29]:
# col_list = onet.columns.tolist()
# select = fnmatch.filter(col_list, '*mana*')
# select

In [33]:
df_linked_leadership.to_csv('UKB_follow-up_job_leadership_2.csv')


In [ ]:
# the variable that should keep: leadership; leadership_d (dummy); wcontext_respon; wstyle_lead; 
